# Simple deep_autoencoder - Mnist

## Import Mnist data

In [1]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print (x_train.shape)
print (x_test.shape)

Using TensorFlow backend.


AttributeError: module 'numpy.core.umath' has no attribute 'divmod'

## Deep_autoencoder structure

In [ ]:
# from keras.layers import Input, Dense
# from keras.models import Model
# from keras import regularizers

# # this is the size of our encoded representations
# encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# input_img = Input(shape=(784,))
# encoded = Dense(128, activation='relu')(input_img)
# encoded = Dense(64, activation='relu')(encoded)
# encoded = Dense(32, activation='relu')(encoded)

# decoded = Dense(64, activation='relu')(encoded)
# decoded = Dense(128, activation='relu')(decoded)
# decoded = Dense(784, activation='sigmoid')(decoded)


## Training Model

In [ ]:
# encoder = Model(input_img, encoded)
# autoencoder=Model(input_img, decoded)

# encoded_dim=Input(shape=(32,))

# decoder= Model(encoded_dim, decoded)
# # decoder = Model(input=encoded_dim, output=autoencoder.layers[-1](autoencoder.layers[-2](autoencoder.layers[-3](encoded_dim))))

# autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# autoencoder.fit(x_train, x_train,
#                 epochs=50,
#                 batch_size=256,
#                 shuffle=True,
#                 validation_data=(x_test, x_test))


## Testing deep_autoencoder prediction

In [ ]:
# encoded_imgs = encoder.predict(x_test)
# # predicted = autoencoder.predict(x_test)

## Ploting

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(40, 4))
# for i in range(10):
#     # display original images
#     ax = plt.subplot(3, 20, i + 1)
#     plt.imshow(x_test[i].reshape(28, 28))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
    
#     # display encoded images
#     ax = plt.subplot(3, 20, i + 1 + 20)
#     plt.imshow(encoded_imgs[i].reshape(8,4))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
    
#     # display reconstructed images
#     ax = plt.subplot(3, 20, 2*20 +i+ 1)
#     plt.imshow(predicted[i].reshape(28, 28))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
  
    
# plt.show()

In [ ]:
from typing import Callable, List, Tuple, Union

from keras.layers import Dense, Input, Layer
from keras import Model


def _architecture_builder(input_shape: Tuple, output_shape: int, neurons: List[int]) -> List[Layer]:
    layers = [Input(input_shape)]
    for n in neurons:
        layers.append(Dense(n, activation='relu'))
    layers.append(Dense(output_shape, activation='sigmoid'))
    return layers


def _model_builder(layers: List[Layer]) -> Model:
    inputs = layers[0]
    layer = layers[1](inputs)
    for l in layers[2:]:
        layer = l(layer)
    model = Model(inputs, layer)
    return model


def dense_encoder_decoder(input_shape: Tuple, output_shape: int, encoder_size: int, neurons: List[int],
                          optimizer: Union[Callable, str] = 'adam', loss: Union[Callable, str] = 'mse')\
        -> Tuple[Model, Model, Model]:

    encoder_layers = _architecture_builder(input_shape, encoder_size, neurons)
    decoder_layers = _architecture_builder((encoder_size,), output_shape, neurons[::-1])
    encoder_decoder_layers = encoder_layers + decoder_layers[1:]
    encoder = _model_builder(encoder_layers)
    decoder = _model_builder(decoder_layers)
    encoder_decoder = _model_builder(encoder_decoder_layers)
    encoder_decoder.compile(optimizer=optimizer, loss=loss)
    return encoder_decoder, encoder, decoder

In [ ]:
autoencoder, encoder, decoder = dense_encoder_decoder((784,), 784, 2, [784,128,64,32])

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))